In [ ]:
from langchain.vectorstores import Chroma

from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.tools import Tool

from langchain.document_loaders import DirectoryLoader, TextLoader

from langchain.memory import ConversationBufferMemory

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder

from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor

from langchain.chains import RetrievalQA

#import env
import os
import sys

In [ ]:
OPENAI_API_KEY = ""

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
loader=TextLoader('analects_raw.txt',encoding='utf-8')
data=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=10)
splits=text_splitter.split_documents(data)
embedding=OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
vectordb=Chroma.from_documents(
    documents=splits,embedding=embedding,collection_name='personal-data'
)
llm=ChatOpenAI(temperature=0)
personal_data=RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',retriever=vectordb.as_retriever()
)

In [ ]:
personal_data_tool=Tool(
    name='PersonalData',
    func=personal_data.run,
    description='논어 데이터'
)
memory_key='history'
memory=ConversationBufferMemory(memory_key=memory_key,return_messages=True)

In [ ]:
system_message=SystemMessage(
    content=(
        '항상 논어 데이터만을 활용해서 논어랑 가장 관련성 높은 대답을 해줘'
        '너가 말한 내용의 근거를 내가 준 논어 데이터에서 찾아서 그대로 말해줘'
        #'논어의 어디서 가져왔는지 출처도 내용과 같이 알려줘.'
    )
)

In [ ]:
prompt=OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
)

In [ ]:
tools=[personal_data_tool]
agent=OpenAIFunctionsAgent(llm=llm,tools=tools,prompt=prompt)
agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
result=agent_executor({'input': '공자님 안녕하세요.'},return_only_outputs=True)
prompt=None

In [ ]:
from langchain.prompts import  PromptTemplate
import pickle

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

with open("conf_styles.pickle", "rb") as pr:
    examples = pickle.load(pr)

In [ ]:
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, OpenAIEmbeddings(), Chroma, k=4
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="""
너는 사용자의 문장을 공자의 말투로 바꿔주는 인공지능이야.
사용자의 질문을 공자라면 어떻게 말했을지 생각해서 같은 의미의 문장을 답변해줘.
반드시 공자의 말투로 줄글로 답변해주어야 해. 
""",
    suffix="Input: {input_noun}\n",
    input_variables=["input_noun"],
)

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = LLMChain(llm=chat_model, prompt=similar_prompt, verbose=True)


In [ ]:
conf_result = chain(similar_prompt.format(input_noun=result['output']))
conf_result['text']

In [ ]:
if len(sys.argv)>1:
    prompt=sys.argv[1]
    
while True:
    if not prompt:
        prompt = input('\033[31m\r\nPrompt: \033[0m')
    if prompt in ['quit','q','exit']:
        sys.exit()
    
    result=agent_executor({'input':prompt},return_only_outputs=True)
    conf_result = chain(similar_prompt.format(input_noun=result['output']))
    print(conf_result["text"])
    
    prompt=None